# OSCordon Quick Start

In [1]:
from lonboard import Map, PathLayer, PolygonLayer

from OSCordon.os_object import OSObject

## Example 1: Single Street with Bounding Box

This example shows how to:

1. Get data for one street using its USRN
2. Create a bound box around it 
3. Display both the street line and bounding box on a map

In [2]:
# Simple Bounding Box and Single USRN
os_obj = OSObject()
usrn = "23009365"
street_data = os_obj.get_street_data(usrn)

bounds = os_obj.create_buffer(street_data, buffer_distance=10, return_geometry=False)

# Convert to WGS84 and return GeoDataFrame
# This has to be specified when working with just 1 USRN if you want the map to show
gdf = os_obj.convert_to_wgs84(street_data, bbox=bounds)

# Create layers for the USRN line and buffer geom
usrn_layer = PathLayer.from_geopandas(
    gdf[gdf["type"] == "usrn"], get_color=[0, 0, 255], width_min_pixels=3
)

boundary_layer = PolygonLayer.from_geopandas(
    gdf[gdf["type"] == "boundary"],
    get_fill_color=[255, 165, 0, 100],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

# Simple Bounding Box
Map(layers=[usrn_layer, boundary_layer])

2025-09-24 23:20:04.165 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:04.167 | SUCCESS  | OSCordon.os_object:create_buffer:344 - Simple Tuple Bounding Box Created Successfully (cap: round, join: round)
2025-09-24 23:20:04.193 | SUCCESS  | OSCordon.os_object:convert_to_wgs84:217 - Converted 2 geometries to WGS84
2025-09-24 23:20:04.194 | DEBUG    | OSCordon.os_object:convert_to_wgs84:218 -        type      usrn street_name  \
0      usrn  23009365        None   
1  boundary  23009365        None   

                                            geometry  
0  MULTILINESTRING ((-1.54705 53.79614, -1.54706 ...  
1  POLYGON ((-1.54689 53.79605, -1.54685 53.79964...  


Map(custom_attribution='', layers=(PathLayer(get_color=[0, 0, 255], table=arro3.core.Table
+------+----------+…

## Example 2: Getting the Buffer Geometry

This shows how to get the actual buffer shape (polygon) instead of just the bounding box coordinates. 

This is useful when you need a precise buffer area for analysis.

Notice that `return_geometry` is set as True for this.

In [3]:
buffer = os_obj.create_buffer(street_data, buffer_distance=10, return_geometry=True)
gdf = os_obj.convert_to_wgs84(street_data, bbox=buffer)

buffer_layer = PolygonLayer.from_geopandas(
    gdf[gdf["type"] == "boundary"],
    get_fill_color=[255, 165, 0, 100],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

Map(layers=[usrn_layer, buffer_layer])

2025-09-24 23:20:15.140 | SUCCESS  | OSCordon.os_object:create_buffer:340 - Complex Boundary Geometry Created Successfully
2025-09-24 23:20:15.143 | SUCCESS  | OSCordon.os_object:convert_to_wgs84:217 - Converted 2 geometries to WGS84
2025-09-24 23:20:15.143 | DEBUG    | OSCordon.os_object:convert_to_wgs84:218 -        type      usrn street_name  \
0      usrn  23009365        None   
1  boundary  23009365        None   

                                            geometry  
0  MULTILINESTRING ((-1.54705 53.79614, -1.54706 ...  
1  POLYGON ((-1.54719 53.79611, -1.54719 53.79611...  


Map(custom_attribution='', layers=(PathLayer(get_color=[0, 0, 255], table=arro3.core.Table
+------+----------+…

## Example 3: Multiple Streets with Bounding Box

This example shows how to:

1. Work with multiple streets at once using a list of USRNs
2. Get a simple bounding box that covers all the streets
3. Display the streets and their overall boundary area

As a reminder, `return_geometry=False` gives you just the roads and a simple rectangle around them, which is good for getting a simple overview of an error or if you need to pass a bbox as a query param to another endpoint.

In [4]:
# Single USRN
usrn_list = ["23012292", "23001950", "23002521"]

# Specify config
config = {
    "cap_style": "round",
    "join_style": "round",
    "resolution": 16,
}

gdf = os_obj.create_connected_route(
    usrn_list, buffer_distance=30, buffer_config=config, return_geometry=False
)

roads = gdf[gdf["type"] == "usrn"]
boundary = gdf[gdf["type"] == "route_bounds"]

# Create layers
road_layer = PathLayer.from_geopandas(
    roads,
    get_color=[0, 0, 255],
    width_min_pixels=3,
)

boundary_layer = PolygonLayer.from_geopandas(
    boundary,
    get_fill_color=[255, 165, 0, 100],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

# Display map
m = Map(layers=[road_layer, boundary_layer])
m

2025-09-24 23:20:23.683 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:23.684 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23012292
2025-09-24 23:20:23.889 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:23.890 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23001950
2025-09-24 23:20:24.079 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:24.079 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23002521
2025-09-24 23:20:24.080 | SUCCESS  | OSCordon.os_object:get_multiple_streets:250 - Retrieved 3 of 3 USRNs
2025-09-24 23:20:24.081 | SUCCESS  | OSCordon.os_object:create_buffer:340 - Complex Boundary Geometry Created Successfully
2025-09-24 23:20:24.082 | SUCCESS  | OSCordon.os_object:create_buffer:340 - Complex Boundary Geometry Created Successfully
2025-09-24 23:20:24.083 | SUCCESS

Map(custom_attribution='', layers=(PathLayer(get_color=[0, 0, 255], table=arro3.core.Table
+------+----------+…

## Example 4: Multiple Streets with Detailed Buffers

This is the same as above, but with `return_geometry=True` to get:
1. Detailed buffer zones that follow the shape of each street
2. A merged buffer area that covers the entire route

This gives you more precise coverage areas.

In [5]:
# Single USRN
usrn_list = ["23012292", "23001950", "23002521"]

# Specify config
config = {
    "cap_style": "round",
    "join_style": "round",
    "resolution": 16,
}

gdf = os_obj.create_connected_route(
    usrn_list,
    buffer_distance=30,
    buffer_config=config,
    merge_buffers=True,
    return_geometry=True,
)

roads = gdf[gdf["type"] == "usrn"]
boundary = gdf[gdf["type"] == "route_buffer"]

# Create layers
road_layer = PathLayer.from_geopandas(
    roads,
    get_color=[0, 0, 255],
    width_min_pixels=3,
)

boundary_layer = PolygonLayer.from_geopandas(
    boundary,
    get_fill_color=[255, 165, 0, 100],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

# Display map
m = Map(layers=[road_layer, boundary_layer])
m

2025-09-24 23:20:29.656 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:29.657 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23012292
2025-09-24 23:20:29.777 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:29.777 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23001950
2025-09-24 23:20:29.978 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:29.979 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23002521
2025-09-24 23:20:29.980 | SUCCESS  | OSCordon.os_object:get_multiple_streets:250 - Retrieved 3 of 3 USRNs
2025-09-24 23:20:29.982 | SUCCESS  | OSCordon.os_object:create_buffer:340 - Complex Boundary Geometry Created Successfully
2025-09-24 23:20:29.982 | SUCCESS  | OSCordon.os_object:create_buffer:340 - Complex Boundary Geometry Created Successfully
2025-09-24 23:20:29.984 | SUCCESS

Map(custom_attribution='', layers=(PathLayer(get_color=[0, 0, 255], table=arro3.core.Table
+------+----------+…

## Example 5: Bounding Box for Dispersed Streets

This shows `create_proximity()` with `return_geometry=False` using streets that are close but not directly connected:
- Returns roads + single bounding box rectangle that encompasses all streets
- Shows how the bounding box covers the entire area even when streets are scattered
- Useful for getting the overall extent of multiple non-connected streets

In [6]:
# Different USRNs - streets that are close but not directly connected
usrn_list_dispersed = ["23001950", "23032833", "23013026"]

# Get simple bounding box (return_geometry=False)
gdf_simple = os_obj.create_proximity(
    usrn_list_dispersed, buffer_distance=30, buffer_config=config, return_geometry=False
)

# Split into layers for simple view
roads_simple = gdf_simple[gdf_simple["type"] == "usrn"]
bounding_box = gdf_simple[gdf_simple["type"] == "route_bounds"]

print(
    f"Dispersed streets - Roads: {len(roads_simple)}, Bounding box: {len(bounding_box)}"
)

# Create map with bounding box
road_layer_simple = PathLayer.from_geopandas(
    roads_simple,
    get_color=[0, 0, 255],
    width_min_pixels=3,
)

bbox_layer = PolygonLayer.from_geopandas(
    bounding_box,
    get_fill_color=[255, 165, 0, 80],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

# Display map with roads and bounding box
m_simple = Map(layers=[road_layer_simple, bbox_layer])
m_simple

2025-09-24 23:20:37.111 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:37.112 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23001950
2025-09-24 23:20:37.263 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:37.264 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23032833
2025-09-24 23:20:37.453 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:37.454 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23013026
2025-09-24 23:20:37.454 | SUCCESS  | OSCordon.os_object:get_multiple_streets:250 - Retrieved 3 of 3 USRNs
2025-09-24 23:20:37.455 | SUCCESS  | OSCordon.os_object:create_buffer:340 - Complex Boundary Geometry Created Successfully
2025-09-24 23:20:37.456 | SUCCESS  | OSCordon.os_object:create_buffer:340 - Complex Boundary Geometry Created Successfully
2025-09-24 23:20:37.456 | SUCCESS

Dispersed streets - Roads: 3, Bounding box: 1


Map(custom_attribution='', layers=(PathLayer(get_color=[0, 0, 255], table=arro3.core.Table
+------+----------+…

## Example 6: Individual Street Buffers

This shows `create_proximity()` with `return_geometry=True` using streets that are close but not directly connected:
- Returns roads + individual buffer polygons for each street
- Shows exact coverage area for each street

In [7]:
usrn_list = ["23001950", "23032833", "23013026"]

config = {
    "cap_style": "round",
    "join_style": "round",
    "resolution": 16,
}

# Get individual buffers for each street (return_geometry=True)
gdf_detailed = os_obj.create_proximity(
    usrn_list, buffer_distance=30, buffer_config=config, return_geometry=True
)

# Split into layers for detailed view
roads_detailed = gdf_detailed[gdf_detailed["type"] == "usrn"]
individual_buffers = gdf_detailed[gdf_detailed["type"] == "buffer"]

print(
    f"Individual buffers - Roads: {len(roads_detailed)}, Buffers: {len(individual_buffers)}"
)

# Create map with individual buffers
road_layer = PathLayer.from_geopandas(
    roads_detailed,
    get_color=[0, 0, 255],
    width_min_pixels=3,
)

buffer_layer = PolygonLayer.from_geopandas(
    individual_buffers,
    get_fill_color=[255, 165, 0, 80],
    get_line_color=[255, 165, 0],
    line_width_min_pixels=2,
)

# Display map with roads and individual buffers
m = Map(layers=[road_layer, buffer_layer])
m

2025-09-24 23:20:41.270 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:41.271 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23001950
2025-09-24 23:20:41.453 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:41.454 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23032833
2025-09-24 23:20:41.652 | SUCCESS  | OSCordon.os_object:_get_single_feature:119 - USRN Data Found
2025-09-24 23:20:41.652 | INFO     | OSCordon.os_object:get_multiple_streets:242 - Retrieved USRN 23013026
2025-09-24 23:20:41.652 | SUCCESS  | OSCordon.os_object:get_multiple_streets:250 - Retrieved 3 of 3 USRNs
2025-09-24 23:20:41.653 | SUCCESS  | OSCordon.os_object:create_buffer:340 - Complex Boundary Geometry Created Successfully
2025-09-24 23:20:41.654 | SUCCESS  | OSCordon.os_object:create_buffer:340 - Complex Boundary Geometry Created Successfully
2025-09-24 23:20:41.655 | SUCCESS

Individual buffers - Roads: 3, Buffers: 3


Map(custom_attribution='', layers=(PathLayer(get_color=[0, 0, 255], table=arro3.core.Table
+------+----------+…